# Machine Learning (Neural Network) Basics
## Part III: Neural Networks (Training)

This notebook is meant to give a brief introduction to the ideas behind machine learning and how they work.

An excellent web-site for playing with simple examples is [playground.tensorflow.org](https://playground.tensorflow.org/).

First we will check to see that all relevant packages have been installed.

In [1]:
# install packages if they're not already
import sys
sys.path.append("figs/")
import os
print ("Using anaconda environment: {}".format(os.environ['CONDA_DEFAULT_ENV']))
!conda install --yes --prefix {sys.prefix} -c numpy scipy matplotlib \
    pandas scikit-learn scikit-learn-intelex pytorch

Using anaconda environment: ml_tutorials
Solving environment: done

# All requested packages already installed.



In [2]:
# we typically always use numpy, pandas and matplotlib, so import those
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# import pytorch
import torch
# determine device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla K20Xm
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Neural Networks (Training)
### Gradient Descent
-----------------------------

Now that we have a grasp on the underlying structure of a neural network, we'd like to have a way of tuning the parameters (the weights) in order to solve the problem we are interested in.  

Most methods for doing this are based on a gradient descent, the most widely used of which (backpropagation) wasn't figured out until 1975 by Werbos who published his result in  (Werbos, P.J. (1975). Beyond Regression: New Tools for Prediction and Analysis in the Behavioral Sciences) which was his PhD thesis.  

The first thing we have to do is decide on a **cost function** which we will try to minimize using a gradient descent method.  The most common choice is to try and minimize the **mean squared error** between the output and the expected output for each sample.  This turns our problem into a convex optimization problem, which is why gradient descent is the logical choice for training.

### Cost Functions
------------------

There are many choices for cost functions, a few are listed below;

**Mean Squared Error** - $\varepsilon(y_{\mathrm{pred}}) = \langle(y_{\mathrm{pred}} - y_{\mathrm{true}})^2\rangle$

**Hinge Loss** - $\varepsilon(y_{\mathrm{pred}}) = \mathrm{max}(0, 1 - y_{\mathrm{true}}\cdot y_{\mathrm{pred}})$

**Logistic Loss** - $\varepsilon(y_{\mathrm{pred}}) = \frac{1}{\ln(2)}\ln\left(1 + e^{-y_{\mathrm{true}}\cdot y_{\mathrm{pred}}}\right)$

**Cross Entropy** - $\varepsilon(y_{\mathrm{pred}}) = -y_{\mathrm{true}}\ln(y_{\mathrm{pred}}) - (1 - y_{\mathrm{true}})\ln\left(1 - y_{\mathrm{pred}}\right)$

The **Mean Squared Error** is a typical choice which is a function of the network parameters (the weights).  What we would like to do is find the global minimum of this function, which will be the minimum error in the network:

<p align="center">
    <img src="figs/parabolic.png">
</p>


### Stochastic, Batch and Mini-Batch Gradient Descent
-----------------------------------------------------

The last piece of the puzzle is the method by which we will adjust the weights.  Once we have a loss function, we have to choose how we are going to use it, which is typically done through a gradient descent method.  

The most basic of these is **Batch Gradient Descent**.  In this method, we evaluate the entire training sample and adjust the weights according to the average error over the whole sample.  For large data sets, this is not an optimum choice since it will take numerous training sessions to converge to the minimum.  

The other extreme is called **Stochastic Gradient Descent**.  The **Stochastic** refers to the fact that we will tune parameters after each of the training samples has been evaluated.  This method will cause the parameters to fluctuate by large amounts in the parameter space since each training sample is pushing and pulling in different directions.  

A more controlled approach is to combine the ideas and use what's called **Mini-Batch Gradient Descent**.  This takes small batches of the training data (10's to 100's or samples) and trains according to the average error of them.  This is a more controlled version of the **Stochastic** method since it averages out over different directions that different samples will pull the parameters in.    

## The Backprop Algorithm
-------------------------

What we would like to do, is construct the algorithm for backpropagation that will tell us how to adjust each of the parameters according to the loss function we chose for our network.  We can attack this problem in many ways, but in general what we want to do is adjust the weights by some small amount based on the input values of the data

$$
\begin{equation}
\omega_i' \approx \omega_i - \alpha\frac{\partial \varepsilon}{\partial \omega_i}
\end{equation}
$$

where $\varepsilon$ is the error function (loss function) and $\alpha$ is a parameter which controls the strength of the change in the parameters called the **Learning Rate**.  This tells us that the change in the weights will be in the direction down the gradient of the error function; $\Delta \omega_i = -\alpha\frac{\partial \varepsilon}{\partial \omega_i}$.  To determine the form of this, let's just take the derivative.  Starting with the output neuron, for the mean square error we have

$$
\begin{align}
\frac{\partial \varepsilon}{\partial \omega_i} &= \frac{\partial}{\partial \omega_i}(y - y_{\mathrm{true}})^2\\
&= 2(y - y_{\mathrm{true}})\frac{\partial y}{\partial \omega_i}
\end{align}
$$

Now remembering our form for the output

$$
\begin{align}
\frac{\partial \varepsilon}{\partial \omega_i}
&= 2(y - y_{\mathrm{true}})\frac{\partial}{\partial \omega_i}\tanh\left(\sum_i \omega_i\cdot x_i + \tilde{\omega}\right)\\
&\approx 2(y - y_{\mathrm{true}})\left(1 - \left(\sum_i \omega_i\cdot x_i + \tilde{\omega}\right)^2\right)\frac{\partial}{\partial \omega_i}\left(\sum_i \omega_i\cdot x_i + \tilde{\omega}\right)
\end{align}
$$

where we used the fact that the derivative of $\tanh(x)$ is approximately $(1 - x^2)$.  Then depending on which parameter you want to change, this just becomes

$$
\begin{equation}
\frac{\partial \varepsilon}{\partial \omega_i} = 2(y - y_{\mathrm{true}})\left(1 - \left(\sum_i \omega_i\cdot x_i + \tilde{\omega}\right)^2\right)x_i
\end{equation}
$$

Thus for the output layer, the weights get adjusted by

$$
\begin{equation}
\Delta \omega_i = -\alpha2(y - y_{\mathrm{true}})\left(1 - \left(\sum_i \omega_i\cdot x_i + \tilde{\omega}\right)^2\right)x_i
\end{equation}
$$

For the other neurons in different layers this formula is slightly different, since we have to adjust based on what the expected output should've been for those neurons.  What we do is use the fact that the error function of a previous neuron is a function of the output neurons error;

$$
\begin{equation}
\frac{\partial \varepsilon}{\partial x_j} = \frac{\partial \varepsilon(y)}{\partial x_j}
\end{equation}
$$

This allows us to do the following

$$
\begin{equation}
\frac{\partial \varepsilon(y)}{\partial x_j} = \frac{\partial \varepsilon}{\partial f(x_j)}\frac{\partial f(x_j)}{\partial x_j} = \frac{\partial \varepsilon}{\partial y}\frac{\partial y}{\partial f(x_j)}\omega_j
\end{equation}
$$

where $f(x_j)$ is the activation function for neuron $x_j$.  To get a clearer picture we have the following

<p align="center">
    <img src="figs/simple_backprop.png">
</p>

So, backpropagation must begin at the output node (since it's the only thing we initially have a $y_{\mathrm{true}}$ for) and then goes 'back' through the network layer by layer adjusting the weights according to the error on the output.  

One of the drawbacks of SGD, is that it can get stuck in a local minimum.  We can see this from the following example where the initial value of the parameters is randomly chosen.

<p align="center">
    <img src="figs/local_min.png">
</p>

### Momentum and Adaptive Algorithms
------------------------------------

Methods such as **Stochastic Gradient Descent** can take a while to converge towards a minimum, mostly because it oscillates in different directions after each training session.  We can control this motion by basically giving the parameters some inertia or momentum$^{18}$.  This means adding a term that depends on the previous value of the change in the parameters.  We are essentially giving weight to the previous value of the gradient, by saying that we should stray too far from where we were headed.  Thus the new value of the weights after two iterations becomes

\begin{equation}
\omega_i'' = \omega_i' - \alpha\frac{\partial \varepsilon}{\partial y'} - \gamma \frac{\partial \varepsilon}{\partial y}
\end{equation}

where $\gamma$ is called the momentum and is usually set to a value close to 1; $\gamma \approx 0.9$. 

### Nesterov, Adagrad, AdaDelta and Adam
----------------------------------------

Besides adding a momentum term, there are many improvements to gradient descent algorithms that have been made over the past few decades.  I will just briefly mention a few here.  

**Nesterov Accelerated Gradient**$^{19}$ - This method makes use of the momentum term added to SGD, but shifts the gradient term a little further based on where the algorithm 'thinks' the new parameters will be once the momentum term is calculed.  This ammounts to analyzing the gradient term with respect to the expected new position, rather than the current one.

\begin{equation}
\omega_i'' = \omega_i' - \alpha\frac{\partial \varepsilon}{\partial y''} - \gamma\frac{\partial \varepsilon}{\partial y} = \omega_i' - \alpha\frac{\partial \varepsilon}{\partial\left(y' - \gamma\frac{\partial \varepsilon}{\partial y}\right)} - \gamma\frac{\partial \varepsilon}{\partial y}
\end{equation}

This gives us an extra push in the direction of the previous gradient and happens to converge faster towards the minimum.  The next level of abstraction comes with making the learning rate dynamical, and to also make it a function of each parameter.

**Adagrad**$^{20}$ - This method makes the learning rate independent for each parameter while also making it dynamical.  It's adjusted based on the amount of change that's happening for each parameter.  For parameters that are changing a lot, **Adagrad** suppresses the change so that it happens more slowely.  For parameters which are not updated very much, the learning rate is increased.  This is achieved by adding the following factor to the learning rate

\begin{equation}
\omega_i' = \omega_i - \frac{\alpha}{\sqrt{G_{ii} + \epsilon}}\frac{\partial \varepsilon}{\partial y}
\end{equation}

where the $G_{ii}$ is a diagonal matrix, whose diagonal elements $(i,i)$ are the sum of the squares of the gradients of each parameter $\omega_i$ up to the previous time step $N$.

\begin{equation}
G_{ii} = \sum_{t=1}^N \left(\frac{\partial \varepsilon}{\partial y_t}\right)^2
\end{equation}

and also where $\epsilon$ is a small factor that keeps the denominator from blowing up when $G_{ii} = 0$.  The learning rate is now adjusted according to the past history of the changes in the parameters.  While this is a nice added feature, it also causes networks to slow down dramatically after long training sessions.  To avoid this problem, we have to make some improvements to **Adagrad**.  There are several, one of which is **AdaDelta**.

#### ADADELTA
-------------
[**ADADELTA: An Adaptive Learning Rate Method**](https://arxiv.org/abs/1212.5701) by M. Zeiler.

This algorithm replaces not only the learning rate $\alpha$, but the sum of the mean square errors $G_{ii}$ with something more reasonable. Instead of keeping track of all of the past gradient terms, it would suffice to just keep track of the last few, or even better by just keeping track of the root-mean-squared error and adjust the learning rate according to the current value plus a momentum term

\begin{equation}
E\left[\left(\frac{\partial \varepsilon}{\partial y'}\right)^2\right]' = \gamma E\left[\left(\frac{\partial \varepsilon}{\partial y}\right)^2\right] + (1 - \gamma)\left(\frac{\partial \varepsilon}{\partial y'}\right)^2
\end{equation}

where $E[y']$ is the current square-error which we will replace the $G_{ii}$ term in the parameter updates with

\begin{equation}
\omega_i' = \omega_i - \frac{\alpha}{\sqrt{E[y'] + \epsilon}}\frac{\partial \varepsilon}{\partial y} = \omega_i - \frac{\alpha}{RMS[y']}\frac{\partial \varepsilon}{\partial y}
\end{equation}

One final trick is to replace $\alpha$ with another term that depends on the change in the parameters, then we will never have to specify a learning rate; it will be determined completely by the problem at hand.  To do this, we consider the square error in the parameters themselves

\begin{equation}
E\left[(\Delta \omega_i')^2\right] = \gamma E\left[(\Delta \omega_i)^2\right] + (1 - \gamma)(\Delta \omega_i')^2
\end{equation}

The root-mean-square of this error is given by

\begin{equation}
RMS[\Delta \omega] = \sqrt{E\left[(\Delta \omega_i')^2\right] + \epsilon}
\end{equation}

And so the definition of the **AdaDelta** algorithm is given by

\begin{equation}
\omega_i'' = \omega_i - \frac{RMS[\Delta \omega]}{RMS[y']}\frac{\partial \varepsilon}{\partial y}
\end{equation}

and we no longer have to choose a learning rate $\alpha$.  This also fixes the problem of training slowing down as the matrix elements $G_{ii}$ get large since the running average $E[y']$ is adjusted accordingly.  The final adaptive algorithm we will look at is **Adam**.

#### ADAM
---------
[**Adam: A Method for Stocahstic Optimization**](https://arxiv.org/abs/1412.6980) by D. Kingma and J. Ba.

This is an **Adaptive Moment Estimation** algorithm which essentially adapts the learning to the first and second moments of the gradients $\frac{\partial \varepsilon}{\partial y}$.  This keeps a running average of the first and second moments through

\begin{align}
\mu' &= \gamma_1 \mu + (1 - \gamma_1) \frac{\partial \varepsilon}{\partial y}\\
\sigma' &= \gamma_2\sigma + (1 - \gamma_2)\frac{\partial \varepsilon}{\partial y}
\end{align}

There is an inherent bias in these definitions to tend towards zero for both moments.  To get around this, we define a bias corrected set of moments

\begin{align}
\tilde{\mu}' &= \frac{\mu'}{1 - \gamma_1'}\\
\tilde{\sigma}' &= \frac{\sigma'}{1 - \gamma_2'}
\end{align}

and the change in the parameters is now given by

\begin{equation}
\omega_i' = \omega_i - \frac{\tilde{\mu}'}{\sqrt{\tilde{\sigma}'} + \epsilon}\frac{\partial \varepsilon}{\partial y}
\end{equation}

The factors $\gamma_1$ and $\gamma_2$ are adjusted after each iteration along with the running averages of the first and second moments.  

To get a feel for how these algorithms perform we will do a simple comparison.

<p align="center">
    <img src="figs/gradient_1.gif">
</p>

<p align="center">
    <img src="figs/gradient_2.gif">
</p>

These two figures show a comparison between the learning rates of the different optimizers, this was taken from a [webpage tutorial](http://sebastianruder.com/optimizing-gradient-descent/).